In [1]:
import ROOT

file = ROOT.TFile( "reco_characteristics/reco_characteristics_version1.root", "READ" )

Welcome to JupyROOT 6.22/06


In [2]:
file.cd( "2017_preTS2/multi rp-0" )
file.ls()

TFile**		reco_characteristics/reco_characteristics_version1.root	
 TFile*		reco_characteristics/reco_characteristics_version1.root	
  TDirectoryFile*		2017_preTS2	2017_preTS2
   TDirectoryFile*		multi rp-0	multi rp-0
    KEY: TDirectoryFile	xi;1	xi
   KEY: TDirectoryFile	single rp-2;1	single rp-2
   KEY: TDirectoryFile	single rp-3;1	single rp-3
   KEY: TDirectoryFile	single rp-23;1	single rp-23
   KEY: TDirectoryFile	single rp-102;1	single rp-102
   KEY: TDirectoryFile	single rp-103;1	single rp-103
   KEY: TDirectoryFile	single rp-123;1	single rp-123
   KEY: TDirectoryFile	multi rp-0;1	multi rp-0
   KEY: TDirectoryFile	multi rp-1;1	multi rp-1
  KEY: TDirectoryFile	2016_preTS2;1	2016_preTS2
  KEY: TDirectoryFile	2016_postTS2;1	2016_postTS2
  KEY: TDirectoryFile	2017_preTS2;1	2017_preTS2
  KEY: TDirectoryFile	2017_postTS2;1	2017_postTS2
  KEY: TDirectoryFile	2018_preTS1;1	2018_preTS1
  KEY: TDirectoryFile	2018_TS1_TS2;1	2018_TS1_TS2
  KEY: TDirectoryFile	2018_postTS2;1	2018_postTS2


In [ ]:
import numpy as np

def get_reco_characteristics( file, period, arm=None, rp=None ):
    
    print ( file )
    print ( period )
    print ( arm )
    print ( rp )
    
    algo_str_ = None
    str_ = None
    if arm is None and rp is not None:
        algo_str_ = "single rp"
        str_ = "{}/{}-{}/xi/".format( period, algo_str_, rp)
    elif arm is not None and rp is None:
        algo_str_ = "multi rp"
        str_ = "{}/{}-{}/xi/".format( period, algo_str_, arm)
    print ( str_ )
    
    g_bias_vs_xi_ = file.Get( str_ + "g_bias_vs_xi" )
    g_resolution_vs_xi_ = file.Get( str_ + "g_resolution_vs_xi" )
    g_systematics_vs_xi_ = file.Get( str_ + "g_systematics_vs_xi" )

    bias_X = np.array( list( g_bias_vs_xi_.GetX() ) )
    bias_Y = np.array( list( g_bias_vs_xi_.GetY() ) )
    resolution_X = np.array( list( g_resolution_vs_xi_.GetX() ) )
    resolution_Y = np.array( list( g_resolution_vs_xi_.GetY() ) )
    systematics_X = np.array( list( g_systematics_vs_xi_.GetX() ) )
    systematics_Y = np.array( list( g_systematics_vs_xi_.GetY() ) )
    
    return ( (bias_X, bias_Y), (resolution_X, resolution_Y), (systematics_X, systematics_Y) )


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

period = "2017_postTS2"
arm = 1

(bias_X, bias_Y), (resolution_X, resolution_Y), (systematics_X, systematics_Y) = get_reco_characteristics( file, period, arm=arm )

print ( systematics_X, systematics_Y )

fig = plt.figure( figsize=(12,10) )
plt.plot( systematics_X, systematics_Y, 'o' )

In [ ]:
import numpy as np

period = "2017_postTS2"

systematics_X = {}
systematics_Y = {}
(bias_X_, bias_Y_), (resolution_X_, resolution_Y_), (systematics_X[0], systematics_Y[0]) = get_reco_characteristics( file, period, arm=0 )
(bias_X_, bias_Y_), (resolution_X_, resolution_Y_), (systematics_X[1], systematics_Y[1]) = get_reco_characteristics( file, period, arm=1 )
#print ( systematics_X, systematics_Y )

for key_ in systematics_X:
    systematics_X[ key_ ] = np.array( systematics_X[ key_ ] )
    systematics_Y[ key_ ] = np.array( systematics_Y[ key_ ] )
    idx_low_edge_ = ( systematics_Y[ key_ ] > 0. ).argmax()
    #idx_high_edge_ = ( systematics_Y[ key_ ].size - ( systematics_Y[ key_ ][::-1] > 0. ).argmax() - 1 )
    idx_low_edge_inv_ = ( systematics_Y[ key_ ][::-1] > 0. ).argmax()
    print ( idx_low_edge_, systematics_Y[ key_ ][ idx_low_edge_ ] )
    #print ( idx_high_edge_, systematics_Y[ key_ ][ idx_high_edge_ ] )
    print ( idx_low_edge_inv_, systematics_Y[ key_ ][::-1][ idx_low_edge_inv_ ] )
    systematics_Y[ key_ ][:idx_low_edge_] = systematics_Y[ key_ ][ idx_low_edge_ ]
    systematics_Y[ key_ ][::-1][:idx_low_edge_inv_] = systematics_Y[ key_ ][::-1][ idx_low_edge_inv_ ]
    
print ( systematics_X, systematics_Y )

fig = plt.figure( figsize=(12,10) )
colors = ( 'lightblue', 'darkblue' )
for idx_, key_ in enumerate( systematics_X ):
    plt.plot( systematics_X[ key_ ], systematics_Y[ key_ ], 'o', color=colors[ idx_ ] )
plt.xlabel( r"$\xi$", fontsize=18 )
plt.ylabel( r"$\Delta\xi$", fontsize=18 )

In [ ]:
def get_systematics_vs_xi( data_periods, fileName="reco_characteristics/reco_characteristics_version1.root" ):
    
    file_systematics_Xi = ROOT.TFile( fileName, "READ" )
    
    aperture_systematics_Xi_period_map = aperture_period_map
    
    systematics_Xi_X = {}
    systematics_Xi_Y = {}
    for idx_, period_ in enumerate( data_periods ):
        systematics_Xi_X[ period_ ] = {}
        systematics_Xi_Y[ period_ ] = {}
        for arm_ in (0,1):
            (bias_X_, bias_Y_), (resolution_X_, resolution_Y_), (systematics_Xi_X_, systematics_Xi_Y_) = get_reco_characteristics( file_systematics_Xi, period=aperture_systematics_Xi_period_map[ period_ ], arm=arm_ )
            systematics_Xi_X[ period_ ][ arm_ ] = systematics_Xi_X_
            systematics_Xi_Y[ period_ ][ arm_ ] = systematics_Xi_Y_
            idx_low_edge_ = ( systematics_Xi_Y[ period_ ][ arm_ ] > 0. ).argmax()
            idx_low_edge_inv_ = ( systematics_Xi_Y[ period_ ][ arm_ ][::-1] > 0. ).argmax()
            print ( idx_low_edge_, systematics_Xi_Y[ period_ ][ arm_ ][ idx_low_edge_ ] )
            print ( idx_low_edge_inv_, systematics_Xi_Y[ period_ ][ arm_ ][::-1][ idx_low_edge_inv_ ] )
            systematics_Xi_Y[ period_ ][ arm_ ][:idx_low_edge_] = systematics_Xi_Y[ period_ ][ arm_ ][ idx_low_edge_ ]
            systematics_Xi_Y[ period_ ][ arm_ ][::-1][:idx_low_edge_inv_] = systematics_Xi_Y[ period_ ][ arm_ ][::-1][ idx_low_edge_inv_ ]
        
    return ( systematics_Xi_X, systematics_Xi_Y )


In [ ]:
from processing import aperture_period_map

data_periods = [ "2017B", "2017C1", "2017C2", "2017D", "2017E", "2017F1", "2017F2", "2017F3" ]

# per period, arm
systematics = {}
systematics[ "Xi" ] = get_systematics_vs_xi( data_periods, fileName="reco_characteristics/reco_characteristics_version1.root" )

systematics

In [ ]:
import h5py

var_ = "Xi"
with h5py.File( "reco_characteristics/reco_characteristics_version1.h5", 'w' ) as f:
    systematics_X_ = systematics[ var_ ][0]
    systematics_Y_ = systematics[ var_ ][1]
    for period_ in systematics_X_:
        for arm_ in systematics_X_[ period_ ]:
            dset_name_ = "{}/{}/{}/X".format( var_, period_, arm_ )
            print ( "Creating dataset {}.".format( dset_name_ ) )
            dset_X = f.create_dataset( dset_name_, data=systematics_X_[ period_ ][ arm_ ] )
            print ( dset_X )
            
            dset_name_ = "{}/{}/{}/Y".format( var_, period_, arm_ )
            print ( "Creating dataset {}.".format( dset_name_ ) )
            dset_Y = f.create_dataset( dset_name_, data=systematics_Y_[ period_ ][ arm_ ] )
            print ( dset_Y )

In [ ]:
with h5py.File( "reco_characteristics/reco_characteristics_version1.h5", 'r' ) as f:
    for key_ in f.keys():
        print ( key_ )
        for period_ in f[ key_ ]:
            print ( period_ )
            for arm_ in f[ key_ ][ period_ ]:
                print ( arm_ )
                print ( list( f[ key_ ][ period_ ][ arm_ ] ) )
                print ( f[ key_ ][ period_ ][ arm_ ][ 'X' ] )
                print ( f[ key_ ][ period_ ][ arm_ ][ 'X' ][:] )
                print ( f[ key_ ][ period_ ][ arm_ ][ 'Y' ] )
                print ( f[ key_ ][ period_ ][ arm_ ][ 'Y' ][:] )